In [ ]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [ ]:
#|default_exp pipes.map.transforms_old

In [ ]:
#|export
# Python native modules
from typing import Callable,Union,TypeVar
# Third party libs
from fastcore.all import *
import torchdata.datapipes as dp
from torchdata.datapipes.map import MapDataPipe
from torchdata.dataloader2.graph import find_dps,DataPipeGraph,Type,DataPipe
# Local modules

# Type Transforms
> DataPipes for calling functions over map based `__getitem__` calls

In [ ]:
#|export
T_co = TypeVar("T_co", covariant=True)

class TypeTransformer(dp.map.MapDataPipe):
    def __init__(
            self,
            # Should allow `__getitem__` and producing elements to be injested by `type_tfms`
            source_datapipe:MapDataPipe[T_co],
            # A list of Callables that accept an input, and return an output
            type_tfms:List[Callable]
    ) -> None:
        self.type_tfms:Pipeline[Callable] = Pipeline(type_tfms)
        self.source_datapipe:MapDataPipe[T_co] = source_datapipe
    
    def __getitem__(self, index) -> T_co:
        data = self.source_datapipe[index]
        return self.type_tfms(data)
            
    def __len__(self) -> int: return len(self.source_datapipe)

TypeTransformer.__doc__ = """On `__getitem__` functions in `self.type_tfms` get called over each element.
Generally `TypeTransformer` as the name suggests is intended to convert elements from one type to another.
reference documentation on how to combine this with `InMemoryCacheHolder`."""

Similar to the `fastrl.pipes.iter.transforms`, `TypeTransformer` can be used to convert elements into
different values...

In [ ]:
add_one = lambda o:o+1
multiple_by_two = lambda o:o*2
pipe = TypeTransformer(range(10),[add_one,multiple_by_two])

results = [pipe[i] for i in range(10)] 
test_eq(results,[2, 4, 6, 8, 10, 12, 14, 16, 18, 20])

However if we want to convert types and keep them, we can combine `TypeTransformer` with `InMemoryCacheHolder`...

In [ ]:
num2str = lambda o:str(o)
add_str_postfix = lambda o:o+'_postfix'
pipe = TypeTransformer(range(10),[num2str,add_str_postfix])
pipe = dp.map.InMemoryCacheHolder(pipe)

results = [pipe[i] for i in range(10)] 
test_eq(results,['0_postfix', '1_postfix', '2_postfix', '3_postfix', 
                 '4_postfix', '5_postfix', '6_postfix', '7_postfix', '8_postfix', '9_postfix'])

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()